In [ ]:
!nvidia-smi

Tue Apr  4 08:10:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re
import copy

In [ ]:
!cp -r "/content/gdrive/My Drive/EVA8_S11_Course_Docs/GPT" "."

In [ ]:
%cd /content/GPT

/content/GPT


In [ ]:
import model_f
import utils_f
from model_f import Transformer
from utils_f import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    encode,
    decode,
    get_batch,
    save_model_to_chekpoint,
    estimate_loss,
)
print(f'    BATCH_SIZE : {BATCH_SIZE}, \
    BLOCK_SIZE : {BLOCK_SIZE}, \
    DEVICE : {DEVICE}, \
    DROPOUT : {DROPOUT}, \
    LEARNING_RATE : {LEARNING_RATE}')
print(f'NUM_EMBED : {NUM_EMBED}, \
    NUM_HEAD : {NUM_HEAD}, \
    NUM_LAYER : {NUM_LAYER}, \
    MAX_ITER : {MAX_ITER},\
    EVAL_INTER {EVAL_INTER}')

    BATCH_SIZE : 32,     BLOCK_SIZE : 20,     DEVICE : cuda,     DROPOUT : 0.2,     LEARNING_RATE : 0.0003
NUM_EMBED : 768,     NUM_HEAD : 6,     NUM_LAYER : 6,     MAX_ITER : 5000,    EVAL_INTER 20


In [ ]:
!pip install transformers
from transformers import AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.9 MB/s eta 0:00:00


In [ ]:
# raw data
# path_do_data = "./data/english_small.txt"
path_do_data = "./data/english.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
print(f'len of data_raw : {len(data_raw)}')
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
print(f'vocab_size : {vocab_size}')
# data_raw = data_raw[4000000:] # short dataset

# train/val splitz
data = encode(text_seq=data_raw, tokenizer=tokenizer)
print(f'len(data) after encoding : {len(data)}')
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
print(f'len(train_data) : {len(train_data)}')
val_data = data[n:]
print(f'len(val_data) : {len(val_data)}')

# train a new model
model = Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
)

# load model to GPU if available

len of data_raw : 163579


Token indices sequence length is longer than the specified maximum sequence length for this model (37443 > 512). Running this sequence through the model will result in indexing errors


vocab_size : 30522
len(data) after encoding : 37443
len(train_data) : 33698
len(val_data) : 3745


In [ ]:
# load model to GPU if available
m = model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)

Model with 89.44M parameters


In [ ]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
MAX_ITER = 500
for step in range(MAX_ITER):
    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=128
        )
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()

step          0 | train loss 10.7223 | val loss 10.7366
step         20 | train loss 4.9287 | val loss 5.6130
step         40 | train loss 3.0605 | val loss 3.9972
step         60 | train loss 2.1312 | val loss 3.3064
step         80 | train loss 1.5846 | val loss 2.6984
step        100 | train loss 1.2129 | val loss 2.3756
step        120 | train loss 0.9703 | val loss 2.2000
step        140 | train loss 0.8288 | val loss 2.0165
step        160 | train loss 0.7013 | val loss 1.8522
step        180 | train loss 0.6050 | val loss 1.8400
step        200 | train loss 0.5662 | val loss 1.8202
step        220 | train loss 0.5012 | val loss 1.7096
step        240 | train loss 0.5089 | val loss 1.7761
step        260 | train loss 0.4691 | val loss 1.7007
step        280 | train loss 0.4537 | val loss 1.6657
step        300 | train loss 0.4419 | val loss 1.5994
step        320 | train loss 0.4395 | val loss 1.6966
step        340 | train loss 0.4160 | val loss 1.6538
step        360 | train lo

In [ ]:
save_model_to_chekpoint(model=m, path_to_checkpoint="checkpoint", epoch=step)

Successfully saved the model to checkpoint/checkpoint_epoch-1999_02.04.2023_05:31:29.pt


In [ ]:
!cp -r '/content/GPT/checkpoint' '/content/gdrive/MyDrive/EVA8_S11_Course_Docs/GPT'

In [ ]:
# generate some output based on the context
for _ in range(5):
  context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
  print(
      decode(
          enc_sec=m.generate(idx=context, max_new_tokens=20, block_size=BLOCK_SIZE)[0],
          tokenizer=tokenizer,
      )
  )

[PAD] to fake vectors vectors bonn good video video video video video video video video video video video video video video
[PAD] problem suffered fast black spread [unused765] directory doesnt repositoryositori survey term weights world problemstu cutout
[PAD] hardware itself999999999999999999
[PAD] man piano weights weights weights weights weights weights weights weights weights weights weights weights weights weights weights weights weights weights
[PAD] were presented web generation steepogt installeduization 88 66 dying do existence understanding strategy distributionming network


In [ ]:
block_size = 5
s_t01 = torch.zeros(block_size, block_size)
print(s_t01)
for T in range(1, 6):
    print(f' T : {T}')
    t01 = (s_t01[:T, :T] == 999).float()
    # t01 = t01.float()
    print('t01 all zeroes tensor')
    print(t01)
    for i in range(T):
        t01[i][i] = 1                    # diag elem wont be masked
        if i > 0:
            t01[i][i-1] = 1              # left of diag elem wont be masked
        if i > 0 and i < T-1:
            t01[i][i+1] = 1              # right of diag elem wont be masked
    print('t01 after diagonal un-masking')
    print(t01)
    wei = torch.full((T, T), 0.1)
    print('initial wei')
    print(wei)
    t01 = t01[:T, :T] == 0                   # Create t01 boolean matrix based on t01 values
    print('t01 boolean showing elements to be masked')
    print(t01)
    wei = wei.masked_fill(t01, float("-inf"))
    print('wei after masking')
    print(wei)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
 T : 1
t01 all zeroes tensor
tensor([[0.]])
t01 after diagonal un-masking
tensor([[1.]])
initial wei
tensor([[0.1000]])
t01 boolean showing elements to be masked
tensor([[False]])
wei after masking
tensor([[0.1000]])
 T : 2
t01 all zeroes tensor
tensor([[0., 0.],
        [0., 0.]])
t01 after diagonal un-masking
tensor([[1., 0.],
        [1., 1.]])
initial wei
tensor([[0.1000, 0.1000],
        [0.1000, 0.1000]])
t01 boolean showing elements to be masked
tensor([[False,  True],
        [False, False]])
wei after masking
tensor([[0.1000,   -inf],
        [0.1000, 0.1000]])
 T : 3
t01 all zeroes tensor
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
t01 after diagonal un-masking
tensor([[1., 0., 0.],
        [1., 1., 1.],
        [0., 1., 1.]])
initial wei
tensor([[0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000],

In [ ]:
import math
math.floor(0.2)

0